# Maintenance Scripts
This notebook helps rename the project from `agentdecompile` to `agentdecompile` and rewrite documentation.

import os
import shutil
import re

def overwrite_file(path, content):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f'Overwrote {path}')

def replace_in_file(path, old, new):
    if not os.path.exists(path): return
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
    if old in content:
        content = content.replace(old, new)
        with open(path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f'Updated {path}')

CONTRIBUTING_MD = """# Contributing to AgentDecompile

Thank you for your interest in improving AgentDecompile! We want to make this the best AI companion for reverse engineers.

## How to Contribute

1.  **Fork** the repository.
2.  **Create a branch** for your feature or fix.
3.  **Code** your changes.
4.  **Test** using `gradle test` and `gradle integrationTest`.
5.  **Submit a Pull Request**.

## Guidelines

-   **Code Style**: We follow standard Java conventions.
-   **Documentation**: Please update docs if you change functionality.
-   **License**: By contributing, you agree that your contributions will be licensed under the project's GNU Affero General Public License v3.0 (AGPL-3.0).
-   **Testing**: Ensure you add unit tests for new features. Integration tests are recommended for Ghidra interactions.

## Development Setup

See [DEVELOPMENT.md](DEVELOPMENT.md) for detailed instructions on setting up your environment.

## Feedback

If you find a bug or have an idea, please [open an issue](https://github.com/bolabaden/AgentDecompile/issues).
"""

DEVELOPMENT_MD = """# Development Guide

This guide covers setting up your development environment for AgentDecompile.

## Prerequisites

-   **Java 21**: We use Java 21 for modern features.
-   **Gradle 8.10+**: Use the system gradle (not wrapper).
-   **Ghidra 12.0+**: Required for the extension.
-   **Python 3.11+**: For the CLI bridge and tests.
-   **uv**: Python package manager.
-   **Git**: Version control.

## Setup

1.  **Clone the repository**:
    ```bash
    git clone https://github.com/bolabaden/AgentDecompile.git
    cd AgentDecompile
    ```

2.  **Environment Variables**:
    Create a `.env` file (optional, or set in shell):
    ```bash
    export GHIDRA_INSTALL_DIR=/path/to/ghidra_12.0_PUBLIC
    ```

3.  **Build**:
    ```bash
    gradle build
    ```

4.  **Install to Ghidra** (Local Dev):
    ```bash
    gradle install
    ```

## Project Structure

-   `src/main/java/agentdecompile`: Core Java source code.
-   `src/agentdecompile_cli`: Python CLI bridge (migrating to `agentdecompile_cli`).
-   `tests`: Python tests.
-   `src/test`: Java unit tests.
-   `src/test.slow`: Java integration tests.

"""

DEVELOPER_MD = """# Developer Notes

Internals and architecture of AgentDecompile.

## Architecture

AgentDecompile runs as a generic Ghidra extension that starts a local HTTP server. This server implements the Model Context Protocol (MCP).

### Key Components

1.  **AgentDecompilePlugin**: The Ghidra plugin that manages the server lifecycle.
2.  **McpServerManager**: Handles MCP requests (list_tools, call_tool, etc.).
3.  **ToolProvider**: Abstract base class for all tools (Decompiler, Listing, etc.).

### Adding a New Tool

1.  Extend `AbstractToolProvider` in `agentdecompile.tools`.
2.  Implement `getToolDef()` to return the JSON schema.
3.  Implement `call()` to handle the request.
4.  Register it in `McpServerManager`.

"""

overwrite_file('CONTRIBUTING.md', CONTRIBUTING_MD)
overwrite_file('DEVELOPMENT.md', DEVELOPMENT_MD)
overwrite_file('DEVELOPER.md', DEVELOPER_MD)

# Rename directories
paths_to_rename = [
    ('src/main/java/agentdecompile', 'src/main/java/agentdecompile'),
    ('src/test/java/agentdecompile', 'src/test/java/agentdecompile'),
    ('src/test.slow/java/agentdecompile', 'src/test.slow/java/agentdecompile'),
    ('src/agentdecompile_cli', 'src/agentdecompile_cli')
]

for old, new in paths_to_rename:
    if os.path.exists(old):
        if os.path.exists(new):
            print(f'Destination {new} exists, merging...')
            # Simple merge logic if needed, or skip
        else:
            os.rename(old, new)
            print(f'Renamed {old} to {new}')

# Recursively replace 'agentdecompile' with 'agentdecompile' in all files
extensions = ['.java', '.py', '.md', '.gradle', '.xml', '.properties', '.toml']

for root, dirs, files in os.walk('.'):
    if '.git' in root or 'build' in root or '.gradle' in root: continue
    
    for file in files:
        if any(file.endswith(ext) for ext in extensions) or file == 'Dockerfile':
            path = os.path.join(root, file)
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                new_content = content
                # Direct code replacements
                new_content = new_content.replace('package agentdecompile', 'package agentdecompile')
                new_content = new_content.replace('import agentdecompile.', 'import agentdecompile.')
                new_content = new_content.replace('agentdecompile.tools', 'agentdecompile.tools')
                
                # General text replacements (case insensitive sometimes?)
                new_content = new_content.replace('AgentDecompile', 'AgentDecompile')
                new_content = new_content.replace('agentdecompile_cli', 'agentdecompile_cli')
                
                # Env vars
                new_content = new_content.replace('AGENTDECOMPILE_', 'AGENT_DECOMPILE_')
                
                if new_content != content:
                    with open(path, 'w', encoding='utf-8') as f:
                        f.write(new_content)
                    print(f'Updated content in {path}')
            except Exception as e:
                print(f'Error processing {path}: {e}')